# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
import numpy as np
import time
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
!pip install scikit-optimize
from skopt import BayesSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [ ]:
df_train = pd.read_csv("https://raw.githubusercontent.com/agateboy/datasetmlpemula/refs/heads/main/Dataset_inisiasi.csv")
df_train

,Price,Sold,Cluster
0,399945,3,3
1,399000,21,3
2,385000,10,3
3,385000,7,3
4,385000,13,3
...,...,...,...
2699,350,47,0
2700,259,1,0
2701,240,635,0
2702,220,297,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [ ]:
X = df_train.drop('Cluster', axis=1)
y = df_train['Cluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Ukuran data latih: {X_train.shape}")
print(f"Ukuran data uji: {X_test.shape}")

Ukuran data latih: (1892, 2)
Ukuran data uji: (812, 2)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [ ]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
initial_mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE) on test set (without tuning): {initial_mse}")

Mean Squared Error (MSE) on test set (without tuning): 0.005550000000000001


Model kali ini dibangun menggunakan algoritma Random Forest Regressor. Nilai MSE pada model regresi ini 0.005 tanpa melakukan hyperparameter tuning. Nilai ini sudah cukup baik dari hasil algoritma ensemble yaitu RandomForestRegressor.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [ ]:
y_pred_classes = np.round(y_pred)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9963054187192119


In [ ]:
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print(f"F1-Score: {f1}")

F1-Score: 0.9960991983000097


In [ ]:
cm = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[592   0   0   0]
 [  0  37   0   0]
 [  0   0 169   0]
 [  0   1   2  11]]


In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       592
           1       0.97      1.00      0.99        37
           2       0.99      1.00      0.99       169
           3       1.00      0.79      0.88        14

    accuracy                           1.00       812
   macro avg       0.99      0.95      0.97       812
weighted avg       1.00      1.00      1.00       812



Pada algoritma ini, akurasi model sudah tinggi yaitu 99.63%. Model sudah berhasil mengklasifikasikan dengan benar 99.63% dari data uji.
F1 Score yang didapatkan dari model ini adalah 99.61% menunjukkan model memiliki keseimbangan yang baik antara precision dan recall, sehingga bisa menghindari false positives dan false negatives.
Intepretasi Confusion Matrix:
*   Kelas 0: Model dengan sempurna mengklasifikasikan semua contoh sebagai kelas 0
*   Kelas 1: Model mengklasifikasikan 37 contoh sebagai kelas 1 dengan benar, namin ada 1 false positives di mana kelas 1 salah diprediksi menjadi kelas 3.
*   Kelas 2: Model mengklasifikasikan 169 contoh sebagai kelas 2 dengan benar.
* Kelas 3: Model mengklasifikasikan 11 contoh dengan benar sebagai kelas 3, tetapi ada 1 false negative di mana kelas 3 sebagai kelas 1 dan 2 kesalahan lain sebagai kelas 2.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [ ]:
start_time = time.time()
param_grid = {
    'n_estimators' : [100, 200, 300],
    'max_depth' : [10, 20, 30],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
    'bootstrap' : [True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print(f"Best parameters (Grid Search): {grid_search.best_params_}")
best_rf_grid = grid_search.best_estimator_
y_pred_grid = best_rf_grid.predict(X_test)
grid_search_mse = mean_squared_error(y_test, y_pred_grid)
print(f"MSE after Grid Search: {grid_search_mse:.2f}")
end_time = time.time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.4f} detik")

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Best parameters (Grid Search): {'bootstrap': False, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
MSE after Grid Search: 0.00
Waktu eksekusi: 147.3132 detik


In [ ]:
start_time = time.time()
param_space = {
    'n_estimators': (100, 500),
    'max_depth': (10, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4),
    'bootstrap': [True, False]
}
bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=32, cv=3, n_jobs=-1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)

print(f"Best parameters (Bayesian Optimization): {bayes_search.best_params_}")
best_rf_bayes = bayes_search.best_estimator_

y_pred_bayes = best_rf_bayes.predict(X_test)
bayes_mse = mean_squared_error(y_test, y_pred_bayes)
print(f"MSE after Grid Search: {bayes_mse:.2f}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Waktu eksekusi: {execution_time:.4f} detik")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
y_pred_grid = best_rf_grid.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_grid)
print(f"Accuracy after Tunning: {accuracy:.4f}")

f1 = f1_score(y_test, y_pred_grid, average='weighted')
print(f"F1-Score after Tunning: {f1:.4f}")

conf_matrix = confusion_matrix(y_test, y_pred_grid)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_test, y_pred_grid)
print("Classification Report:")
print(class_report)

Accuracy after Tunning: 0.9988
F1-Score after Tunning: 0.9988
Confusion Matrix:
[[592   0   0   0]
 [  0  37   0   0]
 [  0   0 169   0]
 [  0   1   0  13]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       592
           1       0.97      1.00      0.99        37
           2       1.00      1.00      1.00       169
           3       1.00      0.93      0.96        14

    accuracy                           1.00       812
   macro avg       0.99      0.98      0.99       812
weighted avg       1.00      1.00      1.00       812



## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Setelah dilakukan tuning, hasil model menunjukkan presisi yang lebih baik. Dengan nilai akurasi dari 99.63% menjadi 99.88% dan F1 Score dari 99,61% menjadi 99.88%. Pada kesalahan prediksi kelas 3 pada awalnya terdapat 2 kesalahan prediksi menjadi kelas 2, setelah dituning hanya terdapat 1 kesalahan terdeteksi menjadi kelas 1 sehingga total prediksi benar adalah 13 pada kelas 3.
Pada kasus ini data yang diberikan sudah menjadi model yang presisi, dan tuning dilakukan untuk meningkatkan nilai prediksinya, yang mana 99% adalah akurasi yang sangat tinggi untuk sebuah model.